In [ ]:
import cv2
import numpy as np

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Función para detectar colores en HSV
def detectar_color(hsv, x, y):
    pixel = hsv[y, x]  # Obtener el color en la posición central del contorno
    h, s, v = pixel

    if (0 <= h <= 10) or (160 <= h <= 180):
        return "Rojo"
    elif 35 <= h <= 85:
        return "Verde"
    elif 90 <= h <= 130:
        return "Azul"
    elif 20 <= h <= 30:
        return "Amarillo"
    else:
        return "Desconocido"

# Función para detectar formas
def detectar_forma(contorno):
    approx = cv2.approxPolyDP(contorno, 0.05 * cv2.arcLength(contorno, True), True)
    vertices = len(approx)

    # Obtener el área y el perímetro
    area = cv2.contourArea(contorno)
    perimetro = cv2.arcLength(contorno, True)

    # Comprobar si es un círculo usando la relación área/perímetro
    if perimetro > 0:
        circularidad = 4 * np.pi * (area / (perimetro ** 2))
        if 0.6 < circularidad <= 1.2:  # Un círculo tiene circularidad cercana a 1
            return "Circulo"

    if vertices == 3:
        return "Triangulo"
    elif vertices == 4:
        return "Cuadrado"
    elif vertices == 5:
        return "Pentagono"
    elif vertices == 6:
        return "Hexagono"
    else:
        return "Desconocido"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir imagen a escala de grises y HSV
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Aplicar desenfoque para reducir ruido
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Detección de bordes
    edges = cv2.Canny(blurred, 50, 150)

    # Encontrar contornos
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contorno in contours:
        area = cv2.contourArea(contorno)
        if area > 500:  # Filtrar ruido y contornos pequeños
            x, y, w, h = cv2.boundingRect(contorno)
            centro_x, centro_y = x + w // 2, y + h // 2  # Punto central del objeto

            # Detectar forma y color
            forma = detectar_forma(contorno)
            color = detectar_color(hsv, centro_x, centro_y)

            # Dibujar el contorno y la etiqueta en la imagen
            cv2.drawContours(frame, [contorno], -1, (0, 255, 0), 2)
            cv2.putText(frame, f"{forma} - {color}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Mostrar la imagen procesada
    cv2.imshow("Detección de Formas y Colores", frame)

    # Presiona 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
